# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt

---
## 2. Load data

In [3]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [4]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [5]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [6]:
# Group off-peak prices by companies and monthrf
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()
# print(monthly_price_by_id)

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
#print(jan_prices)
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as you can!

## Questions to think about

Investigating:
    “I think that the difference between off-peak prices in December and January the preceding year could be a significant feature when predicting churn”

- Which columns should I remove? Which ones do I consider relevant to the dataset? Which ones only have 1 unique value?
- Which new features are needed? E.g extracting month, day of month, day of year and year into individual columns
    - create new columns that will help to predict churn more accurately.
- Which columns should be combined to create "better" columns
    - a “better” column could be a column that improves the accuracy of the model.
    - columns that share very similar info can/should be combined
- Which datasets to combine
    - a column that features in both datasets that share the same values should be joined on.

First merge info I've been given.

In [7]:
df = pd.merge(df, diff, on='id')
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,9.084737e-04,2.086294,99.530517,44.235794,2.086425,9.953056e+01,4.423670e+01,1,0.020057,3.700961
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000000e+00,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000e+00,0,-0.003767,0.177779
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000000e+00,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000e+00,0,-0.004670,0.177779
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000000e+00,0.000000,0.000000,0.000000,0.000003,0.000000e+00,0.000000e+00,0,-0.004547,0.177779
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,4.860000e-10,0.000000,0.000000,0.000000,0.000011,2.896760e-06,4.860000e-10,0,-0.006192,0.162916


## Prices

From the question, I'm trying to find the price differences btw January - December say '23. So I will have to find the average prices in each of the months within and including that time period. 

### Average price changes across periods
Getting the average prices and their peak differences will help identify which prices customers are more sensitive to and in which periods.

In [8]:
mean_prices = price_df.groupby(['id']).agg({
    'price_off_peak_var': 'mean',
    'price_peak_var': 'mean',
    'price_mid_peak_var': 'mean',
    'price_off_peak_fix': 'mean',
    'price_peak_fix': 'mean',
    'price_mid_peak_fix': 'mean',
    }).reset_index()

# print(mean_prices)
# gets each individual country's average prices in each peak period before finding their consecutive differences


In [9]:
mean_prices['off_peak_peak_mean_diff_var'] = mean_prices['price_off_peak_var'] - mean_prices['price_peak_var']
mean_prices['peak_mid_peak_mean_diff_var'] = mean_prices['price_peak_var'] - mean_prices['price_mid_peak_var']
mean_prices['off_peak_mid_peak_mean_diff_var'] = mean_prices['price_off_peak_var'] - mean_prices['price_mid_peak_var']

mean_prices['off_peak_peak_mean_diff_fix'] = mean_prices['price_off_peak_fix'] - mean_prices['price_peak_fix']
mean_prices['peak_mid_peak_mean_diff_fix'] = mean_prices['price_peak_fix'] - mean_prices['price_mid_peak_fix']
mean_prices['off_peak_mid_peak_mean_diff_fix'] = mean_prices['price_off_peak_fix'] - mean_prices['price_mid_peak_fix']

In [10]:
cols = ['id',
        'off_peak_peak_mean_diff_var',
        'peak_mid_peak_mean_diff_var',
        'off_peak_mid_peak_mean_diff_var',
        'off_peak_peak_mean_diff_fix',
        'peak_mid_peak_mean_diff_fix',
        'off_peak_mid_peak_mean_diff_fix']

df = pd.merge(df, mean_prices[cols], on='id')
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_mid_peak,churn,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,off_peak_peak_mean_diff_var,peak_mid_peak_mean_diff_var,off_peak_mid_peak_mean_diff_var,off_peak_peak_mean_diff_fix,peak_mid_peak_mean_diff_fix,off_peak_mid_peak_mean_diff_fix
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,4.423670e+01,1,0.020057,3.700961,0.024038,0.034219,0.058257,18.590255,7.450670,26.040925
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000000e+00,0,-0.003767,0.177779,0.142485,0.007124,0.149609,44.311375,0.000000,44.311375
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000000e+00,0,-0.004670,0.177779,0.082090,0.088421,0.170512,44.385450,0.000000,44.385450
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000000e+00,0,-0.004547,0.177779,0.151210,0.000000,0.151210,44.400265,0.000000,44.400265
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,4.860000e-10,0,-0.006192,0.162916,0.020536,0.030773,0.051309,16.275263,8.137629,24.412893


### Max price changes across periods
Getting the max prices across periods and months helps identify how price shifts, which could influence customer churn. 
Large fluctuations in energy/electricity prices could lead to customer dissatisfaction and higher churn rates. By including these max price changes as features, I can better understand if and how customers are sensitive to price shifts,

In [11]:
# aggregate average prices per month by company
mean_prices_monthly = price_df.groupby(['id', 'price_date']).agg({
    'price_off_peak_var': 'mean',
    'price_peak_var': 'mean',
    'price_mid_peak_var': 'mean',
    'price_off_peak_fix': 'mean',
    'price_peak_fix': 'mean',
    'price_mid_peak_fix': 'mean',
    }).reset_index()

In [12]:
mean_prices_monthly['off_peak_peak_mean_diff_var'] = mean_prices_monthly['price_off_peak_var'] - mean_prices_monthly['price_peak_var']
mean_prices_monthly['peak_mid_peak_mean_diff_var'] = mean_prices_monthly['price_peak_var'] - mean_prices_monthly['price_mid_peak_var']
mean_prices_monthly['off_peak_mid_peak_mean_diff_var'] = mean_prices_monthly['price_off_peak_var'] - mean_prices_monthly['price_mid_peak_var']

mean_prices_monthly['off_peak_peak_mean_diff_fix'] = mean_prices_monthly['price_off_peak_fix'] - mean_prices_monthly['price_peak_fix']
mean_prices_monthly['peak_mid_peak_mean_diff_fix'] = mean_prices_monthly['price_peak_fix'] - mean_prices_monthly['price_mid_peak_fix']
mean_prices_monthly['off_peak_mid_peak_mean_diff_fix'] = mean_prices_monthly['price_off_peak_fix'] - mean_prices_monthly['price_mid_peak_fix']

In [13]:
# calculating the max monthly difference across time periods
max_monthly_periods_diff = mean_prices_monthly.groupby(['id']).agg({
    'off_peak_peak_mean_diff_var': 'max',
    'peak_mid_peak_mean_diff_var': 'max',
    'off_peak_mid_peak_mean_diff_var': 'max',
    'off_peak_peak_mean_diff_fix': 'max',
    'peak_mid_peak_mean_diff_fix': 'max',
    'off_peak_mid_peak_mean_diff_fix': 'max'
}).reset_index().rename(
    columns= {
    'off_peak_peak_mean_diff_var': 'off_peak_peak_max_monthly_diff_var',
    'peak_mid_peak_mean_diff_var': 'peak_mid_peak_max_monthly_diff_var',
    'off_peak_mid_peak_mean_diff_var': 'off_peak_mid_peak_max_monthly_diff_var',
    'off_peak_peak_mean_diff_fix': 'off_peak_peak_max_monthly_diff_fix',
    'peak_mid_peak_mean_diff_fix': 'peak_mid_peak_max_monthly_diff_fix',
    'off_peak_mid_peak_mean_diff_fix': 'off_peak_mid_peak_max_monthly_diff_fix'
})

In [14]:
cols= ['id',
       'off_peak_peak_max_monthly_diff_var',
       'peak_mid_peak_max_monthly_diff_var',
       'off_peak_mid_peak_max_monthly_diff_var',
       'off_peak_peak_max_monthly_diff_fix',
       'peak_mid_peak_max_monthly_diff_fix',
       'off_peak_mid_peak_max_monthly_diff_fix']

df = pd.merge(df, max_monthly_periods_diff[cols], on='id')
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,off_peak_mid_peak_mean_diff_var,off_peak_peak_mean_diff_fix,peak_mid_peak_mean_diff_fix,off_peak_mid_peak_mean_diff_fix,off_peak_peak_max_monthly_diff_var,peak_mid_peak_max_monthly_diff_var,off_peak_mid_peak_max_monthly_diff_var,off_peak_peak_max_monthly_diff_fix,peak_mid_peak_max_monthly_diff_fix,off_peak_mid_peak_max_monthly_diff_fix
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.058257,18.590255,7.450670,26.040925,0.060550,0.085483,0.146033,44.266930,8.145775,44.26693
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.149609,44.311375,0.000000,44.311375,0.151367,0.085483,0.151367,44.444710,0.000000,44.44471
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.170512,44.385450,0.000000,44.385450,0.084587,0.089162,0.172468,44.444710,0.000000,44.44471
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.151210,44.400265,0.000000,44.400265,0.153133,0.000000,0.153133,44.444710,0.000000,44.44471
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,0.051309,16.275263,8.137629,24.412893,0.022225,0.033743,0.055866,16.291555,8.145775,24.43733


## Tenure
Looking at the active time of clients before exploring the peak prices between those periods.

In [15]:
# i used date_renewal - date_activ in prev submission but i realize now that is wrong 
# this is because renewal doesn't tell EXACTLY when the contract ends
df['tenure'] = ((df['date_end'] - df['date_activ'])/ np.timedelta64(1, 'D') / 365.25).astype(int)
# divide by 365.25 to account for leap years
# df['tenure'].head()

In [16]:
df.groupby(['tenure']).agg({'churn': 'mean'}).sort_values(by='churn', ascending=False)

,churn
tenure,
3,0.143713
2,0.133080
4,0.125756
13,0.095238
5,0.085425
12,0.083333
6,0.080713
7,0.073394
11,0.063584


The results suggest that churn rates are generally higher for clients with shorter tenures (2-5 years) and decrease as the tenure increases, especially after 8 years. This pattern indicates that customers who stay longer tend to be more loyal, while newer clients are more likely to churn.

## Transforming dates into months
The difference in churn rates show a large jump between 4 and 5 years (4%). Finding the monthly specific rates within that year can show more details of the influence on a client churning.
To ensure consistency in my time calculations I will set a reference date of 2016-01-01. This is because the dataset contains activation, modification, and end dates for customers, which seem to be around the year 2016. Using this reference date close to the most recent dates ensures that the months calculated will be more meaningful.

- months_activ = Number of months active until reference date (Jan 2016)
- months_end = Number of months of the contract left until reference date (Jan 2016)
- months_modif_prod = Number of months since last modification until reference date (Jan 2016)
- months_renewal = Number of months since last renewal until reference date (Jan 2016)


In [17]:
def convert_months(reference_date, df, column):
    """
    Input a column with timedeltas and return months
    """
    # calculate time difference 
    time_delta = reference_date - df[column]
    # convert days to months
    months = ((time_delta / np.timedelta64(1, 'D')) / 30.44).astype(int)
    return months

In [18]:
# create reference date
reference_date = datetime(2016, 1, 1)

# create converted columns
df['months_activ'] = convert_months(reference_date, df, 'date_activ')
df['months_end'] = convert_months(reference_date, df, 'date_end')
df['months_modif_prod'] = convert_months(reference_date, df, 'date_modif_prod')
df['months_renewal'] = convert_months(reference_date, df, 'date_renewal')

A seasoned active client would be more likely to stay and thus would have a higher value in`months_activ`. A newer client would have a lower value.

The `months_to_end` feature could provide valuable insights into churn patterns and client behaviors based on their contract timing.
- As clients approach the end of their contract, they may seek better deals or consider renewing their contract. New clients might have an exit option if they’re dissatisfied within a specified period. In contrast, clients in the middle of their contract may face penalties for leaving, which can discourage churn during that phase. 

The `months_modif_prod` feature can provide insights to how involded a client is wiith PowerCo. Since this value points to the level of customer service the client has received for them to update or change their existing service.

A client that has recently renewed their contract with PowerCo shows that they are engaged, and that they are loyal to PowerCo. Thus `months_renewal` could be a very important feature.

In [19]:
# drop the datetime columns that I used for feature engineering, cause I no longer need it
remove = [
    'date_activ',
    'date_end',
    'date_modif_prod',
    'date_renewal'
]

df = df.drop(columns=remove)

In [20]:
df['months_end'] = df['months_end'].abs()
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,...,peak_mid_peak_max_monthly_diff_var,off_peak_mid_peak_max_monthly_diff_var,off_peak_peak_max_monthly_diff_fix,peak_mid_peak_max_monthly_diff_fix,off_peak_mid_peak_max_monthly_diff_fix,tenure,months_activ,months_end,months_modif_prod,months_renewal
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,0.00,0,0.0,1.78,0.114481,...,0.085483,0.146033,44.266930,8.145775,44.26693,3,30,5,2,6
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,189.95,0,0.0,16.27,0.145711,...,0.085483,0.151367,44.444710,0.000000,44.44471,7,76,7,76,4
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,47.96,0,0.0,38.72,0.165794,...,0.089162,0.172468,44.444710,0.000000,44.44471,6,68,3,68,8
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,240.04,0,0.0,19.83,0.146694,...,0.000000,0.153133,44.444710,0.000000,44.44471,6,69,2,69,9
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,445.75,526,0.0,131.73,0.116900,...,0.033743,0.055866,16.291555,8.145775,24.43733,6,71,2,71,9


## Transform Boolean & Categorical data
`has_gas` - t/f, `channel_sales` - channel categories , `origin_up` - campaign categories

### has_gas

In [21]:
# replacing 't' with 1 and 'f' with 0
pd.set_option('future.no_silent_downcasting', True)

df['has_gas'] = df['has_gas'].replace({'t': 1, 'f': 0})

In [22]:
df.groupby(['has_gas']).agg({'churn': 'mean'}).sort_values(by='churn', ascending=False)

,churn
has_gas,
0,0.100544
1,0.081856


Customers who are not also gas clients are more susceptible to churn. Customers who are also gas clients of PowerCo, show that they have multiple products and are a loyal customer to the brand. Customers who do not buy gas are almost 2% more likely to churn than customers who also buy gas so it's a useful feature to look at in predicting the model.

### channel_sales

In [23]:
# convert sales codes to categorical type
df['channel_sales'] = df['channel_sales'].astype('category')
# number of values per sales code
df['channel_sales'].value_counts()


channel_sales
foosdfpfkusacimwkcsosbicdxkicaua    6754
MISSING                             3725
lmkebamcaaclubfxadlmueccxoimlema    1843
usilxuppasemubllopkaafesmlibmsdf    1375
ewpakwlliwisiwduibdlfmalxowmwpci     893
sddiedcslfslkckwlfkdpoeeailfpeds      11
epumfxlbckeskwekxbiuasklxalciiuu       3
fixdbufsefwooaasfcxdxadsiekoceaa       2
Name: count, dtype: int64

There are 8 distinct categories of sales channel, so 8 dummy variables can be created for them (OneHot). However, the last 3 categories show that they only have 11, 3 and 2 occurrences respectively. Considering that the dataset has about 14000 rows, this means that these dummy variables will be almost entirely 0 and so will not add much predictive power to the model at all.

For this reason, I will drop these 3 dummy variables.

In [24]:
df = pd.get_dummies(df, columns=['channel_sales'], prefix='channel')
df = df.drop(columns=['channel_sddiedcslfslkckwlfkdpoeeailfpeds', 'channel_epumfxlbckeskwekxbiuasklxalciiuu', 'channel_fixdbufsefwooaasfcxdxadsiekoceaa'])
df.head()

,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,tenure,months_activ,months_end,months_modif_prod,months_renewal,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf
0,24011ae4ebbe3035111d65fa7c15bc57,0,54946,0,0.00,0,0.0,1.78,0.114481,0.098142,...,3,30,5,2,6,False,False,True,False,False
1,d29c2c54acc38ff3c0614d0a653813dd,4660,0,0,189.95,0,0.0,16.27,0.145711,0.000000,...,7,76,7,76,4,True,False,False,False,False
2,764c75f661154dac3a6c254cd082ea7d,544,0,0,47.96,0,0.0,38.72,0.165794,0.087899,...,6,68,3,68,8,False,False,True,False,False
3,bba03439a292a1e166f80264c16191cb,1584,0,0,240.04,0,0.0,19.83,0.146694,0.000000,...,6,69,2,69,9,False,False,False,True,False
4,149d57cf92fc41cf94415803a877cb4b,4425,0,526,445.75,526,0.0,131.73,0.116900,0.100015,...,6,71,2,71,9,True,False,False,False,False


In [25]:
# get_dummies didn't seem to transform completely so will manually replace them with 0s & 1s
df = df.replace({True: 1, False: 0})
df.head()

,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,tenure,months_activ,months_end,months_modif_prod,months_renewal,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf
0,24011ae4ebbe3035111d65fa7c15bc57,0,54946,0,0.00,0,0.0,1.78,0.114481,0.098142,...,3,30,5,2,6,0,0,1,0,0
1,d29c2c54acc38ff3c0614d0a653813dd,4660,0,0,189.95,0,0.0,16.27,0.145711,0.000000,...,7,76,7,76,4,1,0,0,0,0
2,764c75f661154dac3a6c254cd082ea7d,544,0,0,47.96,0,0.0,38.72,0.165794,0.087899,...,6,68,3,68,8,0,0,1,0,0
3,bba03439a292a1e166f80264c16191cb,1584,0,0,240.04,0,0.0,19.83,0.146694,0.000000,...,6,69,2,69,9,0,0,0,1,0
4,149d57cf92fc41cf94415803a877cb4b,4425,0,526,445.75,526,0.0,131.73,0.116900,0.100015,...,6,71,2,71,9,1,0,0,0,0


### origin_up

In [26]:
# convert campaign codes to categorical type
df['origin_up'] = df['origin_up'].astype('category')
# number of values per sales code
df['origin_up'].value_counts()

origin_up
lxidpiddsbxsbosboudacockeimpuepw    7097
kamkkxfxxuwbdslkwifmmcsiusiuosws    4294
ldkssxwpmemidmecebumciepifcamkci    3148
MISSING                               64
usapbepcfoloekilkwsdiboslwaxobdp       2
ewxeelcelemmiwuafmddpobolfuxioce       1
Name: count, dtype: int64

There are 6 distinct categories of origin, the last 3 categories show that they only have 64, 2 and 1 occurrences respectively. Considering that the dataset has about 14000 rows, this means that these dummy variables will be almost entirely 0 and so will not add much predictive power to the model at all.

For this reason, I will drop these 3 dummy variables.

In [27]:
df = pd.get_dummies(df, columns=['origin_up'], prefix='origin')
df = df.drop(columns=['origin_MISSING', 'origin_usapbepcfoloekilkwsdiboslwaxobdp', 'origin_ewxeelcelemmiwuafmddpobolfuxioce'])
df.head()

,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,months_modif_prod,months_renewal,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf,origin_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_ldkssxwpmemidmecebumciepifcamkci,origin_lxidpiddsbxsbosboudacockeimpuepw
0,24011ae4ebbe3035111d65fa7c15bc57,0,54946,0,0.00,0,0.0,1.78,0.114481,0.098142,...,2,6,0,0,1,0,0,False,False,True
1,d29c2c54acc38ff3c0614d0a653813dd,4660,0,0,189.95,0,0.0,16.27,0.145711,0.000000,...,76,4,1,0,0,0,0,True,False,False
2,764c75f661154dac3a6c254cd082ea7d,544,0,0,47.96,0,0.0,38.72,0.165794,0.087899,...,68,8,0,0,1,0,0,True,False,False
3,bba03439a292a1e166f80264c16191cb,1584,0,0,240.04,0,0.0,19.83,0.146694,0.000000,...,69,9,0,0,0,1,0,True,False,False
4,149d57cf92fc41cf94415803a877cb4b,4425,0,526,445.75,526,0.0,131.73,0.116900,0.100015,...,71,9,1,0,0,0,0,True,False,False


In [28]:
df = df.replace({True: 1, False: 0})
df.head()

,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,months_modif_prod,months_renewal,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf,origin_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_ldkssxwpmemidmecebumciepifcamkci,origin_lxidpiddsbxsbosboudacockeimpuepw
0,24011ae4ebbe3035111d65fa7c15bc57,0,54946,0,0.00,0,0.0,1.78,0.114481,0.098142,...,2,6,0,0,1,0,0,0,0,1
1,d29c2c54acc38ff3c0614d0a653813dd,4660,0,0,189.95,0,0.0,16.27,0.145711,0.000000,...,76,4,1,0,0,0,0,1,0,0
2,764c75f661154dac3a6c254cd082ea7d,544,0,0,47.96,0,0.0,38.72,0.165794,0.087899,...,68,8,0,0,1,0,0,1,0,0
3,bba03439a292a1e166f80264c16191cb,1584,0,0,240.04,0,0.0,19.83,0.146694,0.000000,...,69,9,0,0,0,1,0,1,0,0
4,149d57cf92fc41cf94415803a877cb4b,4425,0,526,445.75,526,0.0,131.73,0.116900,0.100015,...,71,9,1,0,0,0,0,1,0,0


## Transform numerical data
From the EDA, I observed skewness in multiple features: 

['cons_12m', 'cons_gas_12m', 'cons_last_month', 'imp_cons', 'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy', 'forecast_meter_rent_12m',  'forecast_price_energy_off_peak', 'forecast_price_energy_peak', 'forecast_price_pow_off_peak']

It is important to transform skewed data because it assumes the data follows [bell curve] normal distribution and can cause negative effects on the model to be built. With skewed data the predictions can be biased and be very inaccurate (focused on noise or outliers). Some ways include:
- square root, cubic root, exponential/power, logarithm, etc 

In [40]:
# skewed data
# in my EDA I stated that the margin columns were skewed which was false. 
# they visually follow a normal (bell curve) distribution
skewed = [
    'cons_12m', 
    'cons_gas_12m', 
    'cons_last_month', 
    'forecast_cons_12m', 
    'forecast_cons_year', 
    'forecast_discount_energy', 
    'forecast_meter_rent_12m',  
    'forecast_price_energy_off_peak', 
    'forecast_price_energy_peak', 
    'forecast_price_pow_off_peak',
    'imp_cons'
    ]

df[skewed].skew()
# there is a mix of high positive and low-moderate negative skewness

cons_12m                           5.997308
cons_gas_12m                       9.597530
cons_last_month                    6.391407
forecast_cons_12m                  7.155853
forecast_cons_year                16.587990
forecast_discount_energy           5.155098
forecast_meter_rent_12m            1.505148
forecast_price_energy_off_peak    -0.119586
forecast_price_energy_peak        -0.014331
forecast_price_pow_off_peak       -4.998772
imp_cons                          13.198799
dtype: float64

In [41]:
df[skewed].describe()
# the std is very high in some which indicates high variability, outliers and noise.

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,imp_cons
count,1.460600e+04,1.460600e+04,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000
mean,1.592203e+05,2.809238e+04,16090.269752,1868.614880,1399.762906,0.966726,63.086871,0.137283,0.050491,43.130056,152.786896
std,5.734653e+05,1.629731e+05,64364.196422,2387.571531,3247.786255,5.108289,66.165783,0.024623,0.049037,4.485988,341.369366
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.674750e+03,0.000000e+00,0.000000,494.995000,0.000000,0.000000,16.180000,0.116340,0.000000,40.606701,0.000000
50%,1.411550e+04,0.000000e+00,792.500000,1112.875000,314.000000,0.000000,18.795000,0.143166,0.084138,44.311378,37.395000
75%,4.076375e+04,0.000000e+00,3383.000000,2401.790000,1745.750000,0.000000,131.030000,0.146348,0.098837,44.311378,193.980000
max,6.207104e+06,4.154590e+06,771203.000000,82902.830000,175375.000000,30.000000,599.310000,0.273963,0.195975,59.266378,15042.790000


In [42]:
# with mostly highly positively skewed data, will be performing a log transformation
# # apply log10 transformation to the high std columns
df['cons_12m'] = np.log10(df["cons_12m"] + 1)
df['cons_gas_12m'] = np.log10(df["cons_gas_12m"] + 1)
df['cons_last_month'] = np.log10(df["cons_last_month"] + 1)
df['forecast_cons_12m'] = np.log10(df["forecast_cons_12m"] + 1)
df['forecast_cons_year'] = np.log10(df["forecast_cons_year"] + 1)
df['forecast_meter_rent_12m'] = np.log10(df["forecast_meter_rent_12m"] + 1)
df['imp_cons'] = np.log10(df["imp_cons"] + 1)

In [43]:
df[skewed].describe()
# the std are now much lower and suitable for predictions

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,imp_cons
count,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000,14606.000000
mean,4.223939,0.779244,2.264646,2.962177,1.784610,0.966726,1.517203,0.137283,0.050491,43.130056,1.240750
std,0.884515,1.717071,1.769305,0.683592,1.584986,5.108289,0.571481,0.024623,0.049037,4.485988,1.138322
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.754023,0.000000,0.000000,2.695477,0.000000,0.000000,1.235023,0.116340,0.000000,40.606701,0.000000
50%,4.149727,0.000000,2.899547,3.046836,2.498311,0.000000,1.296555,0.143166,0.084138,44.311378,1.584275
75%,4.610285,0.000000,3.529430,3.380716,3.242231,0.000000,2.120673,0.146348,0.098837,44.311378,2.289990
max,6.792889,6.618528,5.887169,4.918575,5.243970,30.000000,2.778376,0.273963,0.195975,59.266378,4.177357


## Correlations

It is always useful to look at how correlated all of the features are within your dataset.

This is important because it reveals the linear relationships between features. We want features to correlate with `churn`, as this will indicate that they are good predictors of it. However features that have a very high correlation can sometimes be suspicious. This is because 2 columns that have high correlation indicates that they may share a lot of the same information (the features should be independent of each other). 

For features to be independent, this means that each feature must have absolutely no dependence on any other feature.

Ideally, you want a set of features that have 0 correlation with all of the independent variables (all features except our target variable) and a high correlation with the target variable (churn). However, this is very rarely the case and it is common to have a small degree of correlation between independent features.

Now let's view their correlations.

In [52]:
# need to get just numerical columns = -id
numerical = df.iloc[:, 1:] # all rows without id column

In [53]:
correlation = numerical.corr()

In [63]:
# to view interactive heatmap
import plotly.express as px

In [65]:
fig = px.imshow(
    correlation, 
    labels=dict(color="Correlation"),  # Set a label for the color scale
    x=correlation.columns, 
    y=correlation.columns, 
    color_continuous_scale='RdBu_r',  # Coolwarm equivalent in Plotly
    zmin=-1, zmax=1,  # Set the range for correlation values
    text_auto='.2f'   # Limit decimals
)

# customizing hover template to display information when hovering over squares
fig.update_traces(hovertemplate="Feature 1: %{x}<br>Feature 2: %{y}<br>Correlation: %{z:.2f}")

# Update title and axes
fig.update_layout(
    title='Interactive Correlation Heatmap of Features',
    xaxis_nticks=len(correlation.columns),
    yaxis_nticks=len(correlation.columns),
    title_x=0.5,  # Center the title
    width=800, height=800
)

# Show the plot
fig.update_layout( width=1500, height=1500)
fig.show()


Based on the correlation results I will decide which features to remove and keep.

In [67]:
# num_years_antig highly correlates with independent features: tenure and months_activ
df = df.drop(columns=['num_years_antig'])
df.head()

,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,months_modif_prod,months_renewal,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf,origin_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_ldkssxwpmemidmecebumciepifcamkci,origin_lxidpiddsbxsbosboudacockeimpuepw
0,24011ae4ebbe3035111d65fa7c15bc57,0.000000,4.739944,0.000000,0.000000,0.000000,0.0,0.444045,0.114481,0.098142,...,2,6,0,0,1,0,0,0,0,1
1,d29c2c54acc38ff3c0614d0a653813dd,3.668479,0.000000,0.000000,2.280920,0.000000,0.0,1.237292,0.145711,0.000000,...,76,4,1,0,0,0,0,1,0,0
2,764c75f661154dac3a6c254cd082ea7d,2.736397,0.000000,0.000000,1.689841,0.000000,0.0,1.599009,0.165794,0.087899,...,68,8,0,0,1,0,0,1,0,0
3,bba03439a292a1e166f80264c16191cb,3.200029,0.000000,0.000000,2.382089,0.000000,0.0,1.318689,0.146694,0.000000,...,69,9,0,0,0,1,0,1,0,0
4,149d57cf92fc41cf94415803a877cb4b,3.646011,0.000000,2.721811,2.650065,2.721811,0.0,2.122969,0.116900,0.100015,...,71,9,1,0,0,0,0,1,0,0


In [68]:
# margin_gross and margin_net are perfect correlations of each other
# I want to keep margin_gross as it focuses on the goods consumed without considering other expenses. Since margin_net is the same there are no other expesense to be considered.
df = df.drop(columns=['margin_net_pow_ele'])
df.head()

,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,months_modif_prod,months_renewal,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf,origin_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_ldkssxwpmemidmecebumciepifcamkci,origin_lxidpiddsbxsbosboudacockeimpuepw
0,24011ae4ebbe3035111d65fa7c15bc57,0.000000,4.739944,0.000000,0.000000,0.000000,0.0,0.444045,0.114481,0.098142,...,2,6,0,0,1,0,0,0,0,1
1,d29c2c54acc38ff3c0614d0a653813dd,3.668479,0.000000,0.000000,2.280920,0.000000,0.0,1.237292,0.145711,0.000000,...,76,4,1,0,0,0,0,1,0,0
2,764c75f661154dac3a6c254cd082ea7d,2.736397,0.000000,0.000000,1.689841,0.000000,0.0,1.599009,0.165794,0.087899,...,68,8,0,0,1,0,0,1,0,0
3,bba03439a292a1e166f80264c16191cb,3.200029,0.000000,0.000000,2.382089,0.000000,0.0,1.318689,0.146694,0.000000,...,69,9,0,0,0,1,0,1,0,0
4,149d57cf92fc41cf94415803a877cb4b,3.646011,0.000000,2.721811,2.650065,2.721811,0.0,2.122969,0.116900,0.100015,...,71,9,1,0,0,0,0,1,0,0


In [69]:
# all var_year and var_6m peak period fix prices are perfect correlations for the normal peak period prices
# i want to keep the normal peak period prices as the fix relates only to power prices and that is represented in the normal prices
df = df.drop(columns=[
    'var_year_price_off_peak_fix',
    'var_year_price_peak_fix',
    'var_year_price_mid_peak_fix',
    'var_6m_price_off_peak_fix',
    'var_6m_price_peak_fix',
    'var_6m_price_mid_peak_fix'
])
df.head()

,id,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,...,months_modif_prod,months_renewal,channel_MISSING,channel_ewpakwlliwisiwduibdlfmalxowmwpci,channel_foosdfpfkusacimwkcsosbicdxkicaua,channel_lmkebamcaaclubfxadlmueccxoimlema,channel_usilxuppasemubllopkaafesmlibmsdf,origin_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_ldkssxwpmemidmecebumciepifcamkci,origin_lxidpiddsbxsbosboudacockeimpuepw
0,24011ae4ebbe3035111d65fa7c15bc57,0.000000,4.739944,0.000000,0.000000,0.000000,0.0,0.444045,0.114481,0.098142,...,2,6,0,0,1,0,0,0,0,1
1,d29c2c54acc38ff3c0614d0a653813dd,3.668479,0.000000,0.000000,2.280920,0.000000,0.0,1.237292,0.145711,0.000000,...,76,4,1,0,0,0,0,1,0,0
2,764c75f661154dac3a6c254cd082ea7d,2.736397,0.000000,0.000000,1.689841,0.000000,0.0,1.599009,0.165794,0.087899,...,68,8,0,0,1,0,0,1,0,0
3,bba03439a292a1e166f80264c16191cb,3.200029,0.000000,0.000000,2.382089,0.000000,0.0,1.318689,0.146694,0.000000,...,69,9,0,0,0,1,0,1,0,0
4,149d57cf92fc41cf94415803a877cb4b,3.646011,0.000000,2.721811,2.650065,2.721811,0.0,2.122969,0.116900,0.100015,...,71,9,1,0,0,0,0,1,0,0


I have observed other perfect correlations between features, but have decided to leave them as they tell different information e.g `mean_diff_fix` and `max_monthly_diff_fix`. As they tell different information they may be necessary in different ways for the prediction model for churn. 

Let me view the final heatmap after dropping the columns above, before moving forward.

In [70]:
reduced = df.iloc[:, 1:]
reduced_correlation = reduced.corr()

In [71]:
fig = px.imshow(
    reduced_correlation, 
    labels=dict(color="Correlation"),  
    x=reduced_correlation.columns, 
    y=reduced_correlation.columns, 
    color_continuous_scale='RdBu_r',  
    zmin=-1, zmax=1,  
    text_auto='.2f'  
)

fig.update_traces(hovertemplate="Feature 1: %{x}<br>Feature 2: %{y}<br>Correlation: %{z:.2f}")

fig.update_layout(
    title='Reduced Correlation Heatmap of Features',
    xaxis_nticks=len(reduced_correlation.columns),
    yaxis_nticks=len(reduced_correlation.columns),
    title_x=0.5,  # Center the title
    width=800, height=800
)

fig.update_layout( width=1500, height=1500)
fig.show()